<h2 align="center">Libraries</h2>

In [8]:
import os
import csv
import requests
import pandas as pd
import subprocess

<h2 align="center">Downloads Movie Trailers</h2>
This downloads a movie's trialer from Youtube using the MovieLens dataset of trailer URLS.<br>
yt-dlp is used for downloads, stores movie_name, file_name in the trailerData.csv file

In [ ]:
trailer_amount = 80 #maybe take this amount as input

directoryy = "trailers"
os.makedirs(directoryy, exist_ok=True)
data_csv = "trailerData.csv"
if os.path.exists(data_csv):
    existing_df = pd.read_csv(data_csv)
else:
    existing_df = pd.DataFrame(columns=["file_name", "movie_name", "genre", "transcript"])



df = pd.read_csv("trailer_address.csv")

df = df.head(trailer_amount)
rows_to_append = []
for idx, row in df.iterrows():
    video_id = row['youtubeId']
    title = row['title'].replace(" ", "_").replace("/", "_")
    file_name = f"{title}_{video_id}.mp4"
    output_path = os.path.join(directoryy, file_name)
# no duplicate entries
    if not ((existing_df['file_name'] == file_name).any()):
        if not os.path.exists(output_path):
            print(f"Downloading {title}...")
            command = f'yt-dlp -f mp4 -o "{output_path}" https://www.youtube.com/watch?v={video_id}'
            subprocess.run(command, shell=True)
            if os.path.exists(output_path):#    if download succeeded
                rows_to_append.append([file_name, row['title'], "", ""])
        else:
            print(f"{title} already exists.")
            rows_to_append.append([file_name, row['title'], "", ""])

if rows_to_append:# new rows and saving
    new_df = pd.DataFrame(rows_to_append, columns=["file_name", "movie_name", "genre", "transcript"])
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    updated_df.to_csv(data_csv, index=False)
    print(f"Updated {data_csv} with {len(rows_to_append)} entries.")
else:
    print("No new entries added to trailerData.csv")

print(f"First {trailer_amount} trailers processed.")

<h2 align="center">Scrape Genre's from TMDB</h2>
This scrapes the genre of a movie against it's name from The Movie Data Base(TMDB), then updates the trailerData.csv file

In [9]:
API_KEY_for_genre = '5d603ec4bd433ee159af77e58efdc97d'
TRAILERs_dir = 'trailers'
df_trailersCSV = 'trailerData.csv'
trls_URLS = 'trailer_address.csv'

def get_movie_genres(movie_title):
    """Fetch genres of a movie from TMDb."""
    search_url = 'https://api.themoviedb.org/3/search/movie'
    genre_url = 'https://api.themoviedb.org/3/genre/movie/list'
    genres_response = requests.get(genre_url, params={'api_key': API_KEY_for_genre})
    if genres_response.status_code != 200:
        return ''
    genre_dict = {g['id']: g['name'] for g in genres_response.json().get('genres', [])}
    response = requests.get(search_url, params={'api_key': API_KEY_for_genre, 'query': movie_title})
    if response.status_code != 200 or not response.json().get('results'):
        return ''
    genre_ids = response.json()['results'][0].get('genre_ids', [])
    return ','.join([genre_dict.get(gid, '') for gid in genre_ids if gid in genre_dict])

def update_empty_genres():
    df = pd.read_csv(df_trailersCSV)
    df['genre'] = df['genre'].astype(str)
    for index, row in df.iterrows():
        if pd.isna(row['genre']) or row['genre'].strip() == '':
            movie_name_raw = row['movie_name']
            movie_name_clean = movie_name_raw.split('(')[0].strip()
            genre = get_movie_genres(movie_name_clean)
            print(f"Updated genre for {movie_name_raw}: {genre}")
            df.at[index, 'genre'] = genre

    df.to_csv(df_trailersCSV, index=False)
    print(" Updated trailerData.csv with missing genres.")

if __name__ == '__main__':
    update_empty_genres()

 Updated trailerData.csv with missing genres.
